# Topics


In [1]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("18-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'61,50\n57,56\n44,69\n39,59\n62,41\n62,63\n39,56\n53,44\n21,4\n49,4\n51,13\n33,52\n69,29\n69,48\n51,53\n47,7\n1,27\n51,6\n6,33\n49,43\n69,47\n47,61\n53,4\n4,1\n45,61\n45,8\n7,27\n25,9\n18,7\n21,7\n66,51\n52,69\n65,49\n35,5\n5,40\n29,7\n19,9\n48,1\n13,33\n8,11\n69,57\n3,10\n48,53\n27,1\n54,59\n53,53\n5,21\n69,58\n66,65\n61,58\n40,47\n51,46\n11,1\n38,69\n68,43\n45,69\n19,3\n5,4\n3,18\n5,23\n35,2\n45,67\n59,64\n37,9\n6,13\n6,17\n49,41\n67,43\n51,47\n51,11\n67,53\n53,67\n11,36\n7,9\n45,49\n53,52\n21,1\n1,29\n19,7\n9,2\n13,1\n7,36\n40,53\n58,47\n44,53\n17,23\n55,56\n61,67\n39,50\n57,10\n45,62\n5,12\n35,3\n3,37\n13,20\n48,65\n67,46\n30,7\n39,55\n70,49\n45,48\n37,1\n15,17\n49,12\n1,26\n46,1\n43,61\n37,45\n55,61\n13,17\n55,1\n33,51\n43,55\n3,13\n54,43\n5,31\n39,57\n49,53\n59,39\n55,64\n59,67\n56,43\n49,5\n9,16\n21,2\n3,36\n44,55\n67,40\n53,66\n51,65\n12,11\n55,60\n65,42\n21,13\n63,62\n64,35\n57,50\n14,13\n51,50\n43,9\n46,67\n58,9\n51,60\n66,37\n7,13\n9,27\n57,63\n47,5\n35,4\n1,2\n52,11\n5,37\n

In [5]:
test_data_raw = r"""5,4
4,2
4,5
3,0
2,1
6,3
2,4
1,5
0,6
3,3
2,6
5,1
1,2
5,5
2,5
6,5
1,4
0,4
6,4
1,1
6,1
1,0
0,5
1,6
2,0"""






def preprocess_data (data):
    blocks = [re.findall(r'-?\d+', row) for row in data.split("\n")]

    return [tuple(map(int, row)) for row in blocks]


test_data = preprocess_data(test_data_raw)

display(test_data)


[(5, 4),
 (4, 2),
 (4, 5),
 (3, 0),
 (2, 1),
 (6, 3),
 (2, 4),
 (1, 5),
 (0, 6),
 (3, 3),
 (2, 6),
 (5, 1),
 (1, 2),
 (5, 5),
 (2, 5),
 (6, 5),
 (1, 4),
 (0, 4),
 (6, 4),
 (1, 1),
 (6, 1),
 (1, 0),
 (0, 5),
 (1, 6),
 (2, 0)]

In [6]:
full_data = preprocess_data(data_raw)
full_data

[(61, 50),
 (57, 56),
 (44, 69),
 (39, 59),
 (62, 41),
 (62, 63),
 (39, 56),
 (53, 44),
 (21, 4),
 (49, 4),
 (51, 13),
 (33, 52),
 (69, 29),
 (69, 48),
 (51, 53),
 (47, 7),
 (1, 27),
 (51, 6),
 (6, 33),
 (49, 43),
 (69, 47),
 (47, 61),
 (53, 4),
 (4, 1),
 (45, 61),
 (45, 8),
 (7, 27),
 (25, 9),
 (18, 7),
 (21, 7),
 (66, 51),
 (52, 69),
 (65, 49),
 (35, 5),
 (5, 40),
 (29, 7),
 (19, 9),
 (48, 1),
 (13, 33),
 (8, 11),
 (69, 57),
 (3, 10),
 (48, 53),
 (27, 1),
 (54, 59),
 (53, 53),
 (5, 21),
 (69, 58),
 (66, 65),
 (61, 58),
 (40, 47),
 (51, 46),
 (11, 1),
 (38, 69),
 (68, 43),
 (45, 69),
 (19, 3),
 (5, 4),
 (3, 18),
 (5, 23),
 (35, 2),
 (45, 67),
 (59, 64),
 (37, 9),
 (6, 13),
 (6, 17),
 (49, 41),
 (67, 43),
 (51, 47),
 (51, 11),
 (67, 53),
 (53, 67),
 (11, 36),
 (7, 9),
 (45, 49),
 (53, 52),
 (21, 1),
 (1, 29),
 (19, 7),
 (9, 2),
 (13, 1),
 (7, 36),
 (40, 53),
 (58, 47),
 (44, 53),
 (17, 23),
 (55, 56),
 (61, 67),
 (39, 50),
 (57, 10),
 (45, 62),
 (5, 12),
 (35, 3),
 (3, 37),
 (13, 20),


In [18]:

def solution (data, verbose=False):
    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str
    
    field_size = data[1]
    blocks = data[0]
    # X,Y coordinate, where 
    #   X is the distance from the left edge of your memory space and 
    #   Y is the distance from the top edge of your memory space

    field = np.full(field_size, '.')
    field[(0,0)] = 'S'
    field[(-1,-1)] = 'E'
    

    bytes_fallen = 1024
    for i in range(min(len(blocks), bytes_fallen)):
        field[blocks[i][::-1]] = '#'

    # print(display_field(field))

    # Dijkstra
    def isValid(shape : tuple, index : tuple):
        return (0 <= index[0] < shape[0] and
                0 <= index[1] < shape[1])

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))
    
    def get_neightbors (array, pos):
        directions = [(1,0), (-1,0), (0,1), (0,-1)]
        symbol = ["v", "^", ">", "<"]
        next_nodes = [tuple_add(pos, direction) for direction in directions
                                                                if isValid(array.shape, tuple_add(pos, direction)) 
                                                                   and array[tuple_add(pos, direction)] in ['.', 'E'] ]
        return next_nodes

    start = (tuple(np.argwhere(field == 'S')[0]))
    end_nodes = tuple(np.argwhere(field == 'E')[0]) 

    # visited nodes -> cannot be in quere anymore
    visited = []
    # nodes to iterate over
    queue = [start]
    # Tracks lowest costs per node
    costs = defaultdict(lambda: float('inf'))
    costs[start] = 0

    while queue:
        # find smallest
        current_node = queue[0]
        # print(current_node)
        lowest_cost = costs[current_node]
        if len(queue) > 1:
            for node in queue[1:]:
                if costs[node] < lowest_cost:
                    current_node = node
                    lowest_cost = costs[node]

        queue.remove(current_node)
        visited.append(current_node)

        nbs = get_neightbors(field, current_node)
        # print("Neighbors", nbs)
        for nb in nbs:
            if nb not in visited:
                if costs[nb] > costs[current_node] +1:
                    costs[nb] = costs[current_node] +1
                    queue.append(nb)


    return costs[end_nodes]

sol = solution((test_data, (7,7)), verbose=True)
print(sol)


inf


In [19]:
sol = solution((full_data, (71,71)))
print(sol)

270


# Part 2

In [41]:

def solution2 (data, verbose=False):
    
    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str
    
    def path_finding(blocks_fallen):
        field_size = data[1]
        blocks = data[0]
        # X,Y coordinate, where 
        #   X is the distance from the left edge of your memory space and 
        #   Y is the distance from the top edge of your memory space

        field = np.full(field_size, '.')
        field[(0,0)] = 'S'
        field[(-1,-1)] = 'E'
        

        bytes_fallen = blocks_fallen
        for i in range(min(len(blocks), bytes_fallen)):
            field[blocks[i][::-1]] = '#'

        # print(display_field(field))

        # Dijkstra
        def isValid(shape : tuple, index : tuple):
            return (0 <= index[0] < shape[0] and
                    0 <= index[1] < shape[1])

        def tuple_add (t1, t2):
            return tuple(map(sum, zip(t1, t2)))
        
        def get_neightbors (array, pos):
            directions = [(1,0), (-1,0), (0,1), (0,-1)]
            symbol = ["v", "^", ">", "<"]
            next_nodes = [tuple_add(pos, direction) for direction in directions
                                                                    if isValid(array.shape, tuple_add(pos, direction)) 
                                                                    and array[tuple_add(pos, direction)] in ['.', 'E'] ]
            return next_nodes

        start = (tuple(np.argwhere(field == 'S')[0]))
        end_nodes = tuple(np.argwhere(field == 'E')[0]) 

        # visited nodes -> cannot be in quere anymore
        visited = []
        # nodes to iterate over
        queue = [start]
        # Tracks lowest costs per node
        costs = defaultdict(lambda: float('inf'))
        costs[start] = 0

        while queue:
            # find smallest
            current_node = queue[0]
            # print(current_node)
            lowest_cost = costs[current_node]
            if len(queue) > 1:
                for node in queue[1:]:
                    if costs[node] < lowest_cost:
                        current_node = node
                        lowest_cost = costs[node]

            queue.remove(current_node)
            visited.append(current_node)

            nbs = get_neightbors(field, current_node)
            # print("Neighbors", nbs)
            for nb in nbs:
                if nb not in visited:
                    if costs[nb] > costs[current_node] +1:
                        costs[nb] = costs[current_node] +1
                        queue.append(nb)


        return costs[end_nodes]
    
    last_solution = data[2]

    upper = len(data[0])
    lower = data[2][0]+1
    memory = {data[2][0]: data[2][1]}

    current  = (upper+lower) // 2
    while lower!=upper:
        steps = path_finding(current)
        memory[current] = steps
        if steps == float("inf"):
            upper = current
        else:
            lower = current +1
        
        current = (upper + lower) // 2

    return data[0][current-1], memory[current-1], memory[current+1]


sol = solution2((test_data, (7,7), (12, 22)), verbose=True)
print(sol)

((6, 1), 24, inf)


In [42]:
sol = solution2((full_data, (71,71), (1024, 270)))
print(sol)

((51, 40), 396, inf)
